In [1]:
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchinfo import summary
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchmetrics.classification import F1Score, Accuracy

In [2]:
## ==> 이미지 데이터 존재하는 폴더 경로 설정
img_dir = r'C:\Users\KDP15\Downloads\Dataset/'
img_train = img_dir + 'Test'
## ==> 이미지 데이터셋 변환 : 지정된 경로 아래
imgDS = ImageFolder(root=img_train, transform=transforms.ToTensor())

In [3]:
from torchvision import datasets, transforms
from torchvision.models import resnet50, ResNet50_Weights

In [4]:
data_transform =transforms.Compose([
    transforms.Resize((224,224)),  # resnet 224,224
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # 
])

In [5]:
train_imgDS = ImageFolder(root=img_dir + 'Train', transform=data_transform)
val_imgDS = ImageFolder(root=img_dir + 'Validation', transform=data_transform)

In [7]:
train_imgDS.classes

['Fake', 'Real']

In [16]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)

In [17]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [18]:
for param in model.parameters():
    param.requires_grad = False

In [19]:
# 이진분류로 바꿔주기
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

In [20]:
for param in model.fc.parameters():
    param.requires_grad = True

In [21]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(DEVICE)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [25]:
EPOCH = 5
BATCH_SIZE =  32
LR = 0.001
crossloss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=LR)
trainDL = DataLoader(train_imgDS, batch_size=BATCH_SIZE, shuffle=True)
valDL= DataLoader(val_imgDS, batch_size=BATCH_SIZE, shuffle=False)

In [29]:
for epoch in range(1,EPOCH+1) :
    model.train()
    
    loss_total =0
    for feature, target in trainDL:
        feature = feature.to(DEVICE)
        target = target.to(DEVICE)
        
        
        pre_y = model(feature)
        
        loss = crossloss(pre_y, target)

        # 성능평가 계산
        loss_total += loss.item()
        
        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # 성능평가 계산
        loss_total += loss.item()

    print(f'[{epoch}/{EPOCH}]\n - TRAIN LOSS : {loss_total/len(trainDL)}')

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for feature, target in valDL:
            feature = feature.to(DEVICE)
            target = target.to(DEVICE)

            pre_val = model(feature)
            _, predicted = torch.max(pre_val, 1) 
            total += target.size(0)
            correct += (predicted == target).sum().item()
    print(f'Validation Accuracy: {100 * correct / total}%')


[1/5]
 - TRAIN LOSS : 0.6857552500663009
Validation Accuracy: 83.51932636704879%
[2/5]
 - TRAIN LOSS : 0.6828499036199214
Validation Accuracy: 83.25555442832504%
[3/5]
 - TRAIN LOSS : 0.6791656092558417
Validation Accuracy: 83.34939636806331%


KeyboardInterrupt: 